# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SEGAEL(ATRKCA OGMNL/SASNQBELD ) T', 'calculationPrice': 'close', 'open': 129.23, 'openTime': 1689829032662, 'openSource': 'clffiioa', 'close': 128.36, 'closeTime': 1637265885741, 'closeSource': 'flaofiic', 'high': 125.69, 'highTime': 1620535690298, 'highSource': 'lrdip1aete5ne i uymdc e', 'low': 126.45, 'lowTime': 1672926434448, 'lowSource': '   cma51leeitydeuirpnde', 'latestPrice': 129.8, 'latestSource': 'Close', 'latestTime': 'February 24, 2021', 'latestUpdate': 1684985439431, 'latestVolume': 114090655, 'iexRealtimePrice': 129.22, 'iexRealtimeSize': 151, 'iexLastUpdated': 1638511579668, 'delayedPrice': 129.26, 'delayedPriceTime': 1639898561777, 'oddLotDelayedPrice': 127.74, 'oddLotDelayedPriceTime': 1680941043644, 'extendedPrice': 128.26, 'extendedChange': -0.36, 'extendedChangePercent': -0.00294, 'extendedPriceTime': 1681144674057, 'previousClose': 130.12, 'previousVolume': 164851965, 'change': -0.53, 'changePercent'

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

35.1

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )

df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,125.20,55.21,N/A
1,AAL,22.56,-1.24,N/A
2,AAP,172.25,24.77,N/A
3,AAPL,128.44,35.26,N/A
4,ABBV,113.42,41.15,N/A
...,...,...,...,...
500,YUM,104.97,31.52,N/A
501,ZBH,168.90,-231.73,N/A
502,ZBRA,530.54,55.81,N/A
503,ZION,57.26,24.66,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
df.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
df = df[df['Price-to-Earnings Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.550,2.4,N/A
1,BIO,621.240,4.73,N/A
2,AIV,5.150,4.94,N/A
3,UNM,27.900,7.34,N/A
4,EBAY,60.110,7.69,N/A
5,ALL,111.390,7.8,N/A
6,AFL,51.550,7.91,N/A
7,CPB,47.150,8.07,N/A
8,CE,147.700,8.59,N/A
9,KR,33.570,8.98,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
position_size = float(input('Investment')) / len(df.index)

Investment10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
df['Number of Shares to Buy'] = df.apply(lambda x: math.floor(position_size / x['Price']),axis=1)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.550,2.4,5056
1,BIO,621.240,4.73,321
2,AIV,5.150,4.94,38834
3,UNM,27.900,7.34,7168
4,EBAY,60.110,7.69,3327
5,ALL,111.390,7.8,1795
6,AFL,51.550,7.91,3879
7,CPB,47.150,8.07,4241
8,CE,147.700,8.59,1354
9,KR,33.570,8.98,5957


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [16]:
symbol = 'A'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data['AAPL']['advanced_stats']['enterpriseValue'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit
print(ev_to_ebitda)

34.2457867616018


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [38]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index=rv_columns
            ),
            ignore_index=True
        )

In [39]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.63,N/A,55.07,N/A,7.99,N/A,7.08,N/A,33.643939,N/A,13.743554,N/A,N/A
1,AAL,21.89,N/A,-1.23,N/A,-1.93,N/A,0.7858,N/A,-4.280180,N/A,2.225619,N/A,N/A
2,AAP,167.42,N/A,25.37,N/A,2.95,N/A,1.12,N/A,11.395706,N/A,2.536069,N/A,N/A
3,AAPL,131.21,N/A,34.82,N/A,31.91,N/A,7.36,N/A,25.976770,N/A,19.440420,N/A,N/A
4,ABBV,109.85,N/A,40.23,N/A,14.69,N/A,4.2,N/A,13.768915,N/A,8.502686,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,105.33,N/A,30.48,N/A,-4.01,N/A,5.9,N/A,23.665633,N/A,10.178781,N/A,N/A
501,ZBH,168.59,N/A,-223.98,N/A,2.85,N/A,4.86,N/A,28.334421,N/A,8.206955,N/A,N/A
502,ZBRA,525.74,N/A,57.31,N/A,12.46,N/A,6.06,N/A,36.962551,N/A,14.388578,N/A,N/A
503,ZION,56.87,N/A,25.2,N/A,1.3,N/A,3.06,N/A,13.431271,N/A,3.472898,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [40]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,238.300,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,257.930,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,9.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,44.920,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.520,N/A,14.36,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.180,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,32.829,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,178.580,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2100.300,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,15.904,N/A,31.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [41]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [42]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [43]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.63,N/A,55.07,85.346535,7.99,73.465347,7.0800,79.80198,33.643939,88.118812,13.743554,79.80198,N/A
1,AAL,21.89,N/A,-1.23,15.049505,-1.93,4.950495,0.7858,7.128713,-4.280180,3.366337,2.225619,9.70297,N/A
2,AAP,167.42,N/A,25.37,49.108911,2.95,39.60396,1.1200,12.970297,11.395706,24.950495,2.536069,10.891089,N/A
3,AAPL,131.21,N/A,34.82,69.50495,31.91,95.841584,7.3600,80.594059,25.976770,80.0,19.440420,92.079208,N/A
4,ABBV,109.85,N/A,40.23,75.841584,14.69,87.128713,4.2000,60.0,13.768915,37.821782,8.502686,54.257426,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,105.33,N/A,30.48,59.306931,-4.01,4.752475,5.9000,74.257426,23.665633,74.455446,10.178781,67.128713,N/A
501,ZBH,168.59,N/A,-223.98,1.188119,2.85,37.920792,4.8600,68.316832,28.334421,83.762376,8.206955,52.079208,N/A
502,ZBRA,525.74,N/A,57.31,86.336634,12.46,84.950495,6.0600,75.247525,36.962551,89.306931,14.388578,82.376238,N/A
503,ZION,56.87,N/A,25.20,48.712871,1.30,13.762376,3.0600,45.742574,13.431271,36.039604,3.472898,16.435644,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,0.755941
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,0.425248
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,0.85198
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,0.286139
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,0.534653
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,0.577228
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,0.666832
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,0.432426


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

Enter the value of your portfolio:1000000


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HPE,9.830,1994,-293.68,0.0118812,0.768699,0.134653,4.757379,0.0455446,2.575722,0.039604,0.0579208
1,FTI,8.870,2210,-0.68,0.134653,0.526579,0.0732673,2.722313,0.00792079,1.745265,0.0178218,0.0584158
2,AAL,13.360,1467,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
3,CCL,15.610,1256,-3.90,0.10297,0.380999,0.0554455,3.751649,0.0237624,3.458441,0.0811881,0.0658416
4,HFC,25.660,764,-26.18,0.0435644,0.716944,0.120792,3.301222,0.0118812,3.693071,0.0891089,0.0663366
5,HPQ,19.140,1024,9.17,0.172277,-23.323972,0.0237624,5.822424,0.0673267,2.516859,0.0356436,0.0747525
6,XRX,17.800,1101,9.35,0.174257,0.640297,0.0990099,3.620185,0.019802,1.604068,0.0138614,0.0767327
7,TPR,16.010,1224,-20.58,0.049505,1.200922,0.231683,3.870569,0.029703,1.149885,0.0019802,0.0782178
8,LB,27.330,717,-10.49,0.0633663,-5.047485,0.0455446,8.144139,0.170297,3.119623,0.0613861,0.0851485
9,SYF,25.300,775,7.77,0.152475,0.971730,0.180198,2.129599,0.0039604,1.604512,0.0158416,0.0881188


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: